# Customer Segmentation using RFM analysis

## Introduction

In this project, the transactional data of a store is analysed. The Recency, Frequency and monetary values were computed for each customer for a period of 12 months. Then, the customer are grouped based on their RFM values. The groups are further analysed and the analysis is used to develop business strategy.   

### Terminology
RFM stands for Recency, Frequency and monetary values. Recency is how recent the customer purchased, usually the variable is in days. Frequency is how many time the customer purchased in a fixed time period. For e.g. Number of purchases in a year. Monetary is the total money spend in a fixed time period. 

### Objectives
1. To make use of RFM values to manually categories customers.
2. One of the main objective is to utilize unsupervised machine learning method, K-Means clustering to make categories.


In [1]:
## necessary imports

import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

In [2]:
# read csv file

customer_data = pd.read_csv('./dataset/online.csv')

In [3]:
customer_data.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


In [4]:
customer_data['InvoiceDate'] = pd.to_datetime(customer_data['InvoiceDate'])
customer_data['InvoiceDate'] = customer_data['InvoiceDate'].apply(lambda x: dt(x.year, x.month, x.day))

In [5]:
customer_data['OrderAmt'] = customer_data['Quantity'] * customer_data['UnitPrice']

In [6]:
customer_data.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,OrderAmt
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom,12.60
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom,1.45
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom,22.50
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom,2.10
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom,17.85


In [7]:
reference_date = max(customer_data['InvoiceDate']) + timedelta(days=1)

In [8]:
RFM = customer_data.groupby(['CustomerID']).agg({'InvoiceDate' : lambda x : (reference_date-max(x)).days,
                                                 'InvoiceNo' : 'count',
                                                 'OrderAmt' : 'sum'
                                                })

In [9]:
RFM.rename(columns = {'InvoiceDate':'Recency',
                  'InvoiceNo': 'Frequency',
                  'OrderAmt':'Monetary'}, inplace=True)

In [28]:
RFM.head()

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,3,27,992.82,4,3,3,433,10
12748,1,967,7522.06,4,3,3,433,10
12749,4,37,813.45,4,3,3,433,10
12820,4,17,268.02,4,2,2,422,8
12822,71,9,146.15,2,1,2,212,5


### Quartile method
The quartile method is applied Recency to create 4 groups. Similarly groups for Frequency and Monetary columns were created.  

In [22]:
r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(RFM['Recency'], 4, labels=r_labels)
RFM['R'] = r_quartiles

TypeError: cannot convert the series to <class 'int'>

In [17]:
f_labels = range(0, 4, 1)
f_quartiles = pd.qcut(RFM['Frequency'], 4, labels=f_labels)
RFM['F'] = int(f_quartiles)

In [18]:
m_labels = range(0, 4, 1)
m_quartiles = pd.qcut(RFM['Monetary'], 4, labels=m_labels)
RFM['M'] = int(m_quartiles)

### RFM score and RFM label
RFM score and RFM label to make groups 

In [23]:
def join_rfm(x): 
    return str(int(x['R'])) + str(int(x['F'])) + str(int(x['M']))

In [24]:
RFM['RFM_Segment'] = RFM.apply(join_rfm, axis=1)
RFM['RFM_Score'] = RFM[['R','F','M']].sum(axis=1)

In [27]:
RFM.groupby('RFM_Score').agg({'Recency':'mean',
                             'Frequency':'mean',
                             'Monetary': ['mean', 'count']})

Recency  Frequency     Monetary      
                 mean       mean         mean count
RFM_Score                                          
1          257.585507   2.043478    28.591652   345
2          177.766764   3.201166    47.967289   343
3          144.178744   4.289855    78.643865   414
4          105.847926   6.391705   149.159885   434
5           83.303896   8.862338   163.673558   385
6           63.774278  12.929134   196.191969   381
7           49.089080  16.706897   330.905980   348
8           33.550964  24.752066   442.724325   363
9           22.045307  39.203883   715.060421   309
10           8.058047  77.203166  1715.483113   379

### Observations

1. There are 10 unique RFM score.
2. Higher the RFM score: lower recency, higher frequency, and higher is mean monetary. Indicate valuable customer.   
3. Lower the RFM score, Recency is higher, frequency and monetary values are low. Indicate Churned customer.

Intutively we can create three groups out of the data.
1. 0 < RFM score < 3; bronze
2. 4 <= RFM score <= 8; silver
3. RFM score >= 9; gold


In [43]:
def group_name(df):
    """
    make groups based of RFM score, boundary values are 3 and 8,  assign label gold, silver, bronze
    
    parameters
    ----------
    df: DataFrame
    dataframe with "RFM_score" column
    
    Return
    ------
    label: str
    name of group label
    """
    if df['RFM_Score']<=3:
        label='bronze'
    elif (df['RFM_Score']<= 8) and (df['RFM_Score'] >=4) :
        label='silver'
    else:
        label='gold'
    return label

In [40]:
RFM['group_name'] = RFM.apply(group_name, axis=1)

In [41]:
RFM.groupby('group_name').agg({'Recency':'mean',
                              'Frequency':'mean',
                              'Monetary':['mean', 'count']})

Recency  Frequency     Monetary      
                  mean       mean         mean count
group_name                                          
bronze      190.137024   3.247731    53.426007  1102
gold         14.340116  60.136628  1266.165363   688
silver       68.848770  13.558870   250.320806  1911

### Observations

1. Gold group provide highest business, the customers are regularly purchasing
2. Bronze group has highest recency value and the customers are higly likely to have churned. 
3. Silver group, the customers frequency is low. The customers in this group may convert to gold. 